# ACCY 570 Final Project
### Fall 2022

## Instructions

The final project contains 24 files: this Jupyter notebook and 23 data files. Twenty of the data files are PDF's containing company annual reports. Two of the data files are text files (*AuditAnalytics.txt* and *Compustat.txt*), and there is also an Excel file (*GatherAuditorData.xlsx*). We packaged these files as a ZIP file. *You must extract these files from the ZIP file in order to work with them.*

You will do some of your work in this Jupyter notebook and some in Tableau. You may create additional cells in the notebook if you wish. 

When you have finished:

1. Save your Tableau file as a Tableau Packaged Workbook (.twbx). This makes it much easier for us to grade.
2. Do *not* combine your files into a ZIP or RAR file.
3. Submit this Jupyter notebook and one Tableau workbook on Canvas.
4. If you are working with a partner, please let me know. 
5. If you submit more than once, we will grade your most recent submission.

In [154]:
## Introduction

Welcome to the final project for ACCY 570. 

The project has two parts. In part 1, you will load, clean, and analyze data about the audit industry. Specifically, you will examine the drivers of the fees charged by auditors. In part 2, you will use text analysis and automation to parse 10-K's in PDF format and compute auditor tenure.

You have learned much this semester and in this project you will bring to bear much of your newfound knowledge and skills. However, when you perform analytics in the real world, you will often need to learn new skills. This project is no different. That's why we also spent time this semester talking about using the documentation for Python commands and using new libraries. You will learn a few things in this project that we did not directly teach you over the semester.

Enjoy!

## Part 1 - Drivers of Audit Fees

In part 1, you will examine the determinants and drivers of audit fees. You will work with real data on audit fees and client characteristics. First, you will load and clean the two datasets containing audit fees and client characteristics. Then you will *merge* these datasets. Finally, you will perform some analyses (e.g., descriptive statistics, scatter plots). 

Following are descriptions of the two datasets with which you will work.

<b><u>Audit Analytics</u></b>  
The *Audit Analytics* database tracks the fees charged by auditing firms. The dataset contains one row per client per year. For example, there is one row for Microsoft's 2018 audit fees. 

<b><u>Compustat</u></b>  
The *Compustat* database tracks companies' accounting fundamentals. *Compustat* contains one row per company per fiscal year. 

### Problem 1 - Loading and Cleaning the Audit Analytics Dataset (50 points)

The Audit Analytics database contains many columns and we downloaded only a subset. Your raw dataset, *AuditAnalytics.txt*, contains these columns:

| Column             | Description                                                       |
|--------------------|-------------------------------------------------------------------|
| AUDITOR_FKEY       | A unique identifier for each auditor                              |
| AUDIT_GIG_KEY      | A unique identifier for each audit engagement                     |
| FISCAL_YEAR        | The fiscal year for this client and this row of data              |
| FISCAL_YEAR_ENDED  | The date on which this client's fiscal year ends                  |
| AUDIT_FEES         | Fees charged by the auditor for the audit                         |
| NON_AUDIT_FEES     | *No description given in the database documentation*              |
| TOTAL_FEES         | Total fees paid by the client to the auditor in this fiscal year  |
| AUDIT_RELATED_FEES | *No description given in the database documentation*              |
| OTHER_FEES         | *No description given in the database documentation*              |
| AUDITOR_NAME       | Name of the auditing firm                                         |
| COMPANY_FKEY       | EDGAR Central Index Key (CIK): a unique identifier for the client |
| BEST_EDGAR_TICKER  | The client's stock ticker                                         |
| NAME               | The client's name                                                 |
| SIC_CODE_FKEY      | 4-digit industry classifier code                                  |

#### Problem 1.1 - Imports (2 points)

Import Pandas and Numpy, as you will use these throughout the project.

In [155]:
# Enter your code in this cell
import pandas as pd
import numpy as np

#### Problem 1.2 - Load raw data file, *AuditAnalytics.txt* (6 points)

The file *AuditAnalytics.txt* contains data from fiscal year 2018. We chose 2018 because it is the last full fiscal year before the COVID-19 pandemic. 

The data file contains __*tab-delimited*__ data. This is like CSV data, but uses tabs to separate the columns instead of commas. You can view this file in Excel if that will help you as you work with it.

---

* Use the Pandas `read_csv` function to load this tab-delimited data into a Pandas data frame. 
* Use a meaningful name for your data frame as you will create other data frames later.
* *You must tell the `read_csv` function that your dataset is tab-delimited. Look up the help for this function and figure out how to do this.*

In [156]:
# Enter your code in this cell
audit_analytics: pd.DataFrame = pd.read_csv(filepath_or_buffer="data/AuditAnalytics.txt", sep="\t")
audit_analytics

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,COMPANY_FKEY,BEST_EDGAR_TICKER,NAME,SIC_CODE_FKEY
0,4,227633,2018,31MAY2018,1988901,317296,2306197,174659,0,KPMG LLP,1750,AIR,AAR CORP,3720.0
1,2,230983,2018,31DEC2018,25381000,9683000,35064000,770000,8000,Ernst & Young LLP,1800,ABT,ABBOTT LABORATORIES,2834.0
2,3143,231837,2018,31DEC2018,12500,12000,24500,10500,0,M&K CPAS PLLC,1961,WDDD,WORLDS INC,7372.0
3,11761,228249,2018,30JUN2018,2028000,229000,2257000,0,0,BDO USA LLP,2034,NaN,ACETO CORP,5122.0
4,8256,231256,2018,31DEC2018,328000,74000,402000,22000,0,Marcum LLP,2098,ACU,ACME UNITED CORP,3420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,3,244483,2018,31DEC2018,2221000,206000,2427000,0,5000,Deloitte & Touche LLP,1793294,NaN,"PPD, Inc.",8731.0
9796,2,244138,2018,31DEC2018,3603000,47000,3650000,0,41000,Ernst & Young LLP,1794338,NaN,International General Insurance Holdings Ltd.,6399.0
9797,4,242381,2018,31DEC2018,1100000,400000,1500000,0,0,KPMG LLP,1794846,ATCO,Atlas Corp.,4412.0
9798,1,246413,2018,31DEC2018,34665,0,34665,0,0,PricewaterhouseCoopers LLP,1796209,APG,APi Group Corp,1700.0


#### Problem 1.3 - Drop unneeded columns (4 points)

The dataset contains some extraneous columns. Drop the columns `SIC_CODE_FKEY` and `BEST_EDGAR_TICKER` as we will get this information from Compustat later.

In [157]:
# Enter your code in this cell
audit_analytics.drop(labels=["SIC_CODE_FKEY", "BEST_EDGAR_TICKER"], axis=1, inplace=True)
audit_analytics

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,COMPANY_FKEY,NAME
0,4,227633,2018,31MAY2018,1988901,317296,2306197,174659,0,KPMG LLP,1750,AAR CORP
1,2,230983,2018,31DEC2018,25381000,9683000,35064000,770000,8000,Ernst & Young LLP,1800,ABBOTT LABORATORIES
2,3143,231837,2018,31DEC2018,12500,12000,24500,10500,0,M&K CPAS PLLC,1961,WORLDS INC
3,11761,228249,2018,30JUN2018,2028000,229000,2257000,0,0,BDO USA LLP,2034,ACETO CORP
4,8256,231256,2018,31DEC2018,328000,74000,402000,22000,0,Marcum LLP,2098,ACME UNITED CORP
...,...,...,...,...,...,...,...,...,...,...,...,...
9795,3,244483,2018,31DEC2018,2221000,206000,2427000,0,5000,Deloitte & Touche LLP,1793294,"PPD, Inc."
9796,2,244138,2018,31DEC2018,3603000,47000,3650000,0,41000,Ernst & Young LLP,1794338,International General Insurance Holdings Ltd.
9797,4,242381,2018,31DEC2018,1100000,400000,1500000,0,0,KPMG LLP,1794846,Atlas Corp.
9798,1,246413,2018,31DEC2018,34665,0,34665,0,0,PricewaterhouseCoopers LLP,1796209,APi Group Corp


#### Problem 1.4 - Filter to 2018 fiscal year (3 points)

The dataset should only contain data for fiscal year 2018. 
1. Verify that the dataset contains only 2018 data. Write some code in the cell below that demonstrates this.
2. If there are rows for other fiscal years, drop those rows.

In [158]:
# Enter your code in this cell
audit_analytics.loc[audit_analytics.FISCAL_YEAR!=2018]
# audit_analytics.drop(index=audit_analytics[audit_analytics.FISCAL_YEAR!=2018].index, inplace=True)

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,COMPANY_FKEY,NAME


#### Problem 1.5 - Date conversion (3 points)

Check the values in the `FISCAL_YEAR_ENDED` column. Write code that shows us whether they contain valid Pandas dates. If not, convert them to Pandas dates.

In [159]:
# Enter your code in this cell
audit_analytics.FISCAL_YEAR_ENDED = pd.to_datetime(audit_analytics.FISCAL_YEAR_ENDED)
audit_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   AUDITOR_FKEY        9800 non-null   int64         
 1   AUDIT_GIG_KEY       9800 non-null   int64         
 2   FISCAL_YEAR         9800 non-null   int64         
 3   FISCAL_YEAR_ENDED   9800 non-null   datetime64[ns]
 4   AUDIT_FEES          9800 non-null   int64         
 5   NON_AUDIT_FEES      9800 non-null   int64         
 6   TOTAL_FEES          9800 non-null   int64         
 7   AUDIT_RELATED_FEES  9800 non-null   int64         
 8   OTHER_FEES          9800 non-null   int64         
 9   AUDITOR_NAME        9800 non-null   object        
 10  COMPANY_FKEY        9800 non-null   int64         
 11  NAME                9800 non-null   object        
dtypes: datetime64[ns](1), int64(9), object(2)
memory usage: 918.9+ KB


#### Problem 1.6 - Descriptive stats of audit fees (2 points)

**Required:** Use the Pandas `describe` method to compute descriptive statistics on the `AUDIT_FEES` column.

**Optional:** The output will be in scientific notation. If you want to make the output easier to read, append this method to your call to the `describe` method. It will format the numbers with commas and without decimals.  
`.map(lambda x: f'{x:,.0f}')`  
This is optional and intended to help you. If you are having difficulty with it, just ignore it.

In [160]:
# Enter your code in this cell
audit_analytics.AUDIT_FEES.describe().map(lambda x: f'{x:,.0f}')

count          9,800
mean       1,619,924
std        4,689,183
min                0
25%           55,000
50%          260,755
75%        1,321,512
max      106,221,720
Name: AUDIT_FEES, dtype: object

#### Problem 1.7 - Truncating the data (8 points)

Looks like there are some large audit fees and some small ones. While the large fees might be considered outliers, they are likely legitimate numbers and should *not* be discarded. For example, it does not make sense to discard rows for large companies like Amazon and General Motors. However, data from very small companies is often "noisy"  and we should consider discarding such data.

* Compute the 5th percentile value of the `AUDIT_FEES` column. 
* Print this 5th percentile value using the `print` function.
* Filter out / remove all rows where the audit fee is strictly less than the 5th percentile value. Note: strictly less (<) does *not* mean less than or equal to (<=).

In [161]:
# Enter your code in this cell
audit_fees_50_quantile: pd.Series = audit_analytics.AUDIT_FEES.quantile([0.5])
print(audit_fees_50_quantile)
audit_analytics.drop(index=audit_analytics[audit_analytics.AUDIT_FEES<audit_fees_50_quantile.values[0]].index, inplace=True)

0.5    260755.0
Name: AUDIT_FEES, dtype: float64


#### Problem 1.8 - Spot checks (6 points)

Let's see if the data makes sense. Search the data and write code that displays or prints the following:
1. Which firm audited Ford Motor Company in 2018?
2. How much did Ford pay in audit fees (AUDIT_FEES)?
3. Which firm audited Illinois Tool Works in 2018?
4. How much did ITW pay in audit fees?

In [162]:
# Enter your code in this cell
audit_ford_info: pd.DataFrame = audit_analytics[audit_analytics.NAME=="FORD MOTOR CO"]
print(f"{audit_ford_info.AUDITOR_NAME.values[0]} audited Ford in 2018, and Ford paid {audit_ford_info.AUDIT_FEES.values[0]} in audit fees.")

audit_itw_info: pd.DataFrame = audit_analytics[audit_analytics.NAME=="ILLINOIS TOOL WORKS INC"]
print(f"{audit_itw_info.AUDITOR_NAME.values[0]} audited ITW in 2018, and ITW paid {audit_itw_info.AUDIT_FEES.values[0]} in audit fees.")

PricewaterhouseCoopers LLP audited Ford in 2018, and Ford paid 37600000 in audit fees.
Deloitte & Touche LLP audited ITW in 2018, and ITW paid 9764000 in audit fees.


#### Problem 1.9 - Non-unique company identifiers (5 points)

The column `COMPANY_FKEY` contains a unique company (client) identifier. Since we are working with data for one year (2018), there should only be one row per company. However, if you check, you will find that there are several repeated `COMPANY_FKEY` values. Verify this. Write code that demonstrates that there are some non-unique `COMPANY_FKEY` values.

In [163]:
# Enter your code in this cell
audit_analytics.COMPANY_FKEY.duplicated().sum()

34

#### Problem 1.10 - Deleting non-unique company identifiers (11 points)

Why might there be non-unique values of `COMPANY_FKEY`? One explanation is that some companies changed auditing firms in the middle of the year and Audit Analytics reports the fees for the former and current auditor. Another explanation is that an auditor performed multiple engagements at the same client (e.g., consulting work) and for some reason this is reported in separate rows. 

Data is never perfect, and data analysis often requires that you clean the data in imperfect ways. Our strategy is this. If a `COMPANY_FKEY` is repeated, then we will keep the row that contains the highest value of AUDIT_FEES. For example, say that `COMPANY_FKEY` 17 is repeated 3 times. In one row, AUDIT_FEES is 10. In the next row, AUDIT_FEES is 20, and in the third row, AUDIT_FEES is 30. You should drop the first two rows and only keep the third row with AUDIT_FEES of 30.

---

You will do this in stages. First, for each `COMPANY_FKEY`, compute the maximum amount of `AUDIT_FEES` paid. Write the code for this first step in the code cell below.

*Hint:* use a pivot table, and save the pivot table as `dfMax`. Then rename the `AUDIT_FEES` column in `dfMax` to `Max_Audit_Fees`.

In [164]:
# Enter your code in this cell
dfMax: pd.DataFrame = audit_analytics.sort_values(by="AUDIT_FEES")
dfMax.drop_duplicates(subset=["COMPANY_FKEY"], keep="last", inplace=True)
dfMax = dfMax[["COMPANY_FKEY", "AUDIT_FEES"]]
dfMax.rename(columns={"AUDIT_FEES": "Max_Audit_Fees"}, inplace=True)
dfMax

,COMPANY_FKEY,Max_Audit_Fees
1492,768847,260810
1965,829323,262000
4130,1099160,262000
8567,1647320,262270
3508,1034594,262500
...,...,...
8160,1610520,71310000
468,67088,72513824
4013,1089113,86600000
302,40545,92200000


Next, you will merge `dfMax` with your Audit Analytics data frame. We did not teach you how to merge data this semester so you will need to teach yourself the basics (and you will learn more about merging data next semester in ACCY 575). 

Here's a simple example to help you get started. Consider this data frame containing some information about pets:

|     | key | Type | Age |
|----:|----:|:-----|----:|
|   0 |   1 | cat  |   4 |
|   1 |   2 | dog  |   9 |
|   2 |   3 | bird |   3 |

Now say there is another data frame with related information:

|     | key | name    |
|----:|----:|:--------|
|   0 |   1 | Fluffy  |
|   1 |   3 | Tweetie |
|   2 |   5 | Spike   |


If we merge these two data frames on the column named "key" with an inner join, we will get this data frame:

|     | key | Type | Age | name    |
|----:|----:|:-----|----:|:--------|
|   0 |   1 | cat  |   4 | Fluffy  |
|   1 |   3 | bird |   3 | Tweetie |

For the rows where the keys matched (keys 1 and 3), the merge appended the columns from both data frames into the resulting data frame. If a key is only in one of the two data frames, its row will not appear in the merged data frame.

---

In the cell below, write code to merge `dfMax` with your Audit Analytics data frame. We strongly suggest that you use the [pd.merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) function. Use an inner join, and join on the `COMPANY_FKEY` column.

In [165]:
# Enter your code in this cell
audit_analytics = pd.merge(left=audit_analytics, right=dfMax, how="inner")
audit_analytics

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,COMPANY_FKEY,NAME,Max_Audit_Fees
0,4,227633,2018,2018-05-31,1988901,317296,2306197,174659,0,KPMG LLP,1750,AAR CORP,1988901
1,2,230983,2018,2018-12-31,25381000,9683000,35064000,770000,8000,Ernst & Young LLP,1800,ABBOTT LABORATORIES,25381000
2,11761,228249,2018,2018-06-30,2028000,229000,2257000,0,0,BDO USA LLP,2034,ACETO CORP,2028000
3,8256,231256,2018,2018-12-31,328000,74000,402000,22000,0,Marcum LLP,2098,ACME UNITED CORP,328000
4,1,230432,2018,2018-12-31,272000,101900,373900,26500,0,PricewaterhouseCoopers LLP,2110,COLUMBIA ACORN TRUST,272000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,1,240882,2018,2018-12-31,2155000,527000,2682000,302000,5000,PricewaterhouseCoopers LLP,1792580,Ovintiv Inc.,2155000
4896,3,244483,2018,2018-12-31,2221000,206000,2427000,0,5000,Deloitte & Touche LLP,1793294,"PPD, Inc.",2221000
4897,2,244138,2018,2018-12-31,3603000,47000,3650000,0,41000,Ernst & Young LLP,1794338,International General Insurance Holdings Ltd.,3603000
4898,4,242381,2018,2018-12-31,1100000,400000,1500000,0,0,KPMG LLP,1794846,Atlas Corp.,1100000


Finally, filter the merged data frame so that you keep rows where the audit fee is equal to the maximum audit fee.

In [166]:
# Enter your code in this cell
audit_analytics.drop(index=audit_analytics[audit_analytics.Max_Audit_Fees!=audit_analytics.AUDIT_FEES].index, inplace=True)
audit_analytics

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,COMPANY_FKEY,NAME,Max_Audit_Fees
0,4,227633,2018,2018-05-31,1988901,317296,2306197,174659,0,KPMG LLP,1750,AAR CORP,1988901
1,2,230983,2018,2018-12-31,25381000,9683000,35064000,770000,8000,Ernst & Young LLP,1800,ABBOTT LABORATORIES,25381000
2,11761,228249,2018,2018-06-30,2028000,229000,2257000,0,0,BDO USA LLP,2034,ACETO CORP,2028000
3,8256,231256,2018,2018-12-31,328000,74000,402000,22000,0,Marcum LLP,2098,ACME UNITED CORP,328000
4,1,230432,2018,2018-12-31,272000,101900,373900,26500,0,PricewaterhouseCoopers LLP,2110,COLUMBIA ACORN TRUST,272000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,1,240882,2018,2018-12-31,2155000,527000,2682000,302000,5000,PricewaterhouseCoopers LLP,1792580,Ovintiv Inc.,2155000
4896,3,244483,2018,2018-12-31,2221000,206000,2427000,0,5000,Deloitte & Touche LLP,1793294,"PPD, Inc.",2221000
4897,2,244138,2018,2018-12-31,3603000,47000,3650000,0,41000,Ernst & Young LLP,1794338,International General Insurance Holdings Ltd.,3603000
4898,4,242381,2018,2018-12-31,1100000,400000,1500000,0,0,KPMG LLP,1794846,Atlas Corp.,1100000


Let's check if our strategy worked. Check again whether there are any repeated COMPANY_FKEY values. If so, print those out.

In [167]:
# Enter your code in this cell
audit_analytics.COMPANY_FKEY.duplicated().sum()

0

Looks like there are still a few repeated `COMPANY_FKEY` values. Let's ignore those and proceed. It will add some noise to our analysis, but hopefully not much.

### Problem 2 - Loading and Cleaning the Compustat Dataset (35 points)

The Compustat database contains hundreds of columns and we downloaded only a subset. Your raw dataset, *Compustat.txt*, contains these columns:

| Column   | Description                                                                                                                     |
|----------|---------------------------------------------------------------------------------------------------------------------------------|
| GVKEY    | A unique identifier for each company                                                                                            |
| DATADATE | The reporting date (fiscal year end date)                                                                                       |
| FYEAR    | Fiscal year for this row                                                                                                        |
| INDFMT   | Industry format (INDL for industrial, FS for financial services)                                                                |
| CONSOL   | Level of consolidation                                                                                                          |
| POPSRC   | Population source (D for domestic (U.S. based) or I for international)                                                          |
| DATAFMT  | Data format                                                                                                                     |
| TIC      | Stock ticker symbol                                                                                                             |
| CONM     | Company name                                                                                                                    |
| CURCD    | Currency code                                                                                                                   |
| FYR      | Fiscal year end month (i.e., 1 for January, 2 for February)                                                                     |
| ACT      | Current assets (in millions of US dollars)                                                                                      |
| AT       | Total assets (in millions of US dollars)                                                                                        |
| CEQ      | Common equity  (in millions of US dollars)                                                                                      |
| IB       | Income before extraordinary items (in millions of US dollars)                                                                   |
| INVT     | Inventories, total (in millions of US dollars)                                                                                  |
| INVWIP   | Inventories, work-in-process (in millions of US dollars)                                                                        |
| LCT      | Current liabilities (in millions of US dollars)                                                                                 |
| LT       | Total liabilities (in millions of US dollars)                                                                                   |
| NI       | Net income (in millions of US dollars)                                                                                          |
| RECT     | Receivables, total (in millions of US dollars)                                                                                  |
| REVT     | SALE plus other operating revenues (in millions of US dollars)                                                                  |
| SALE     | Gross sales less cash discounts, trade discounts, returns, and allowances (in millions of US dollars)                           |
| CIK      | Central Index Key (CIK) is the SEC's unique identifier for corporations and individuals who have filed disclosures with the SEC |
| COSTAT   | Company status (A for active, I for inactive)                                                                                   |
| SIC      | Standard industry classification code                                                                                           |

#### Problem 2.1 - Load raw data file, *Compustat.txt* (3 points)

* Use the Pandas `read_csv` function to load the file *Compustat.txt*, which contains tab-delimited data, into a Pandas data frame.
* Use a meaningful name for your data frame as you will create other data frames later.
* *You must tell the `read_csv` function that your dataset is tab-delimited. Look up the help for this function and figure out how to do this.*

In [168]:
# Enter your code in this cell
compustat: pd.DataFrame = pd.read_csv(filepath_or_buffer="data/Compustat.txt", sep="\t")
compustat

,GVKEY,DATADATE,FYEAR,INDFMT,CONSOL,POPSRC,DATAFMT,TIC,CONM,CURCD,...,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,COSTAT,SIC
0,1004,31MAY2017,2016,INDL,C,D,STD,AIR,AAR CORP,USD,...,25.800,335.100,589.900,56.500,251.400,1767.600,1767.600,1750.0,A,5080.0
1,1004,31MAY2018,2017,INDL,C,D,STD,AIR,AAR CORP,USD,...,32.100,333.300,588.400,15.600,203.000,1748.300,1748.300,1750.0,A,5080.0
2,1004,31MAY2019,2018,INDL,C,D,STD,AIR,AAR CORP,USD,...,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750.0,A,5080.0
3,1019,31DEC2017,2017,INDL,C,D,STD,AFAP,AFA PROTECTIVE SYSTEMS INC,USD,...,NaN,16.338,22.302,0.260,12.689,73.220,73.220,2668.0,A,7380.0
4,1019,31DEC2018,2018,INDL,C,D,STD,AFAP,AFA PROTECTIVE SYSTEMS INC,USD,...,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668.0,A,7380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28036,347085,28FEB2019,2018,INDL,C,D,STD,KARO,KAROOOOO LTD,USD,...,0.908,26.334,49.180,16.851,15.349,120.390,120.390,1828102.0,A,7370.0
28037,348892,31DEC2019,2019,INDL,C,D,STD,STKH,STEAKHOLDER FOODS LTD,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1828098.0,A,3550.0
28038,351038,31DEC2019,2019,INDL,C,D,STD,QNRX,CELLECT BIOTECHNOLOGY LTD,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1671502.0,A,2834.0
28039,351491,31DEC2019,2019,INDL,C,D,STD,IVCGF,IVECO GROUP N V,USD,...,349.000,12588.000,14797.000,94.000,8114.000,13408.000,13408.000,NaN,A,3711.0


#### Problem 2.2 - Drop unneeded columns (2 points)

The data contains many extraneous columns. Drop the columns `INDFMT`, `CONSOL`, `POPSRC`, `DATAFMT`, `CURCD`, and `COSTAT`.

In [169]:
# Enter your code in this cell
compustat.drop(labels=["INDFMT", "CONSOL", "POPSRC", "DATAFMT", "CURCD", "COSTAT"], axis=1, inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
0,1004,31MAY2017,2016,AIR,AAR CORP,5,888.500,1504.100,914.200,50.200,601.100,25.800,335.100,589.900,56.500,251.400,1767.600,1767.600,1750.0,5080.0
1,1004,31MAY2018,2017,AIR,AAR CORP,5,942.700,1524.700,936.300,73.700,547.900,32.100,333.300,588.400,15.600,203.000,1748.300,1748.300,1750.0,5080.0
2,1004,31MAY2019,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750.0,5080.0
3,1019,31DEC2017,2017,AFAP,AFA PROTECTIVE SYSTEMS INC,12,19.724,30.485,8.183,0.260,4.860,NaN,16.338,22.302,0.260,12.689,73.220,73.220,2668.0,7380.0
4,1019,31DEC2018,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668.0,7380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28036,347085,28FEB2019,2018,KARO,KAROOOOO LTD,2,34.196,108.754,43.983,16.851,14.653,0.908,26.334,49.180,16.851,15.349,120.390,120.390,1828102.0,7370.0
28037,348892,31DEC2019,2019,STKH,STEAKHOLDER FOODS LTD,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1828098.0,3550.0
28038,351038,31DEC2019,2019,QNRX,CELLECT BIOTECHNOLOGY LTD,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1671502.0,2834.0
28039,351491,31DEC2019,2019,IVCGF,IVECO GROUP N V,12,11570.000,17847.000,3007.000,94.000,2913.000,349.000,12588.000,14797.000,94.000,8114.000,13408.000,13408.000,NaN,3711.0


#### Problem 2.3 - Filter to 2018 fiscal year (2 points)

The Compustat data file contains multiple fiscal years (you may check if you wish). Since we will eventually merge Compustat with Audit Analytics, we only want 2018 Compustat data. Therefore, filter the Compustat dataset so it only contains data for fiscal year 2018.

In [170]:
# Enter your code in this cell
compustat.drop(index=compustat[compustat.FYEAR!=2018].index, inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,31MAY2019,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750.0,5080.0
4,1019,31DEC2018,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668.0,7380.0
7,1045,31DEC2018,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201.0,4512.0
10,1050,31DEC2018,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197.0,3564.0
13,1062,30NOV2018,2018,ASA,ASA GOLD AND PRECIOUS METALS,11,NaN,196.072,194.834,-48.789,0.000,0.000,NaN,1.238,-48.789,0.103,1.629,1.629,1230869.0,6799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28026,335466,31DEC2018,2018,HOFSQ,HERMITAGE OFFSHORE SERVICES,12,14.160,191.074,54.064,-197.294,1.181,NaN,4.482,137.010,-197.294,2.602,20.654,20.654,1597659.0,NaN
28028,339965,31JAN2019,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147.0,7370.0
28030,345920,31DEC2018,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295.0,3524.0
28033,345980,31DEC2018,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250.0,5961.0


After the filter, make a copy of your data frame by modifying and running the code cell below. For example, if your data frame is named `df`, run the command `df = df.copy()`. Don't worry about why. Just trust us.

In [171]:
compustat = compustat.copy()

#### Problem 2.4 - Date conversion (2 points)

We might need to use the `DATADATE` field later, but it is currently stored as a string. Convert the column to valid Pandas dates.

In [172]:
# Enter your code in this cell
compustat.DATADATE = pd.to_datetime(compustat.DATADATE)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750.0,5080.0
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668.0,7380.0
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201.0,4512.0
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197.0,3564.0
13,1062,2018-11-30,2018,ASA,ASA GOLD AND PRECIOUS METALS,11,NaN,196.072,194.834,-48.789,0.000,0.000,NaN,1.238,-48.789,0.103,1.629,1.629,1230869.0,6799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28026,335466,2018-12-31,2018,HOFSQ,HERMITAGE OFFSHORE SERVICES,12,14.160,191.074,54.064,-197.294,1.181,NaN,4.482,137.010,-197.294,2.602,20.654,20.654,1597659.0,NaN
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147.0,7370.0
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295.0,3524.0
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250.0,5961.0


#### Problem 2.5 - Dealing with missing values (3 points)

Our primary identifier in this dataset is the `CIK` column. We must therefore impose the requirement that `CIK` is not missing. Drop any rows in which the `CIK` column has a missing value.

In [173]:
# Enter your code in this cell
compustat.dropna(subset=["CIK"], inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750.0,5080.0
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668.0,7380.0
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201.0,4512.0
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197.0,3564.0
13,1062,2018-11-30,2018,ASA,ASA GOLD AND PRECIOUS METALS,11,NaN,196.072,194.834,-48.789,0.000,0.000,NaN,1.238,-48.789,0.103,1.629,1.629,1230869.0,6799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28026,335466,2018-12-31,2018,HOFSQ,HERMITAGE OFFSHORE SERVICES,12,14.160,191.074,54.064,-197.294,1.181,NaN,4.482,137.010,-197.294,2.602,20.654,20.654,1597659.0,NaN
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147.0,7370.0
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295.0,3524.0
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250.0,5961.0


We lost about 2,000 rows out of about 9,000 when we dropped those rows! This is common when working with such data. Many small and privately-held companies, and many hedge funds, do not file reports with the SEC and may lack an identifier.

---

We will use SIC codes in a moment. Before we do, let's drop rows with missing values in the `SIC` column:

In [174]:
# Enter your code in this cell
compustat.dropna(subset=["SIC"], inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750.0,5080.0
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668.0,7380.0
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201.0,4512.0
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197.0,3564.0
13,1062,2018-11-30,2018,ASA,ASA GOLD AND PRECIOUS METALS,11,NaN,196.072,194.834,-48.789,0.000,0.000,NaN,1.238,-48.789,0.103,1.629,1.629,1230869.0,6799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28023,332115,2018-12-31,2018,ARMP,ARMATA PHARMACEUTICALS INC,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,921114.0,2836.0
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147.0,7370.0
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295.0,3524.0
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250.0,5961.0


#### Problem 2.6 - Dealing with data types (3 points)

If you check, you will find that Pandas has stored storing the CIK and SIC columns as floats. However, these values cannot contain fractional amounts and therefore it is better to store them as integers. Convert these two columns to the numpy int64 type (`np.int64`).

In [175]:
# Enter your code in this cell
compustat[["CIK"]] = compustat[["CIK"]].astype(np.int64)
compustat[["SIC"]] = compustat[["SIC"]].astype(np.int64)
compustat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7049 entries, 2 to 28036
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   GVKEY     7049 non-null   int64         
 1   DATADATE  7049 non-null   datetime64[ns]
 2   FYEAR     7049 non-null   int64         
 3   TIC       7049 non-null   object        
 4   CONM      7049 non-null   object        
 5   FYR       7049 non-null   int64         
 6   ACT       5125 non-null   float64       
 7   AT        6394 non-null   float64       
 8   CEQ       6378 non-null   float64       
 9   IB        6374 non-null   float64       
 10  INVT      6315 non-null   float64       
 11  INVWIP    4403 non-null   float64       
 12  LCT       5129 non-null   float64       
 13  LT        6389 non-null   float64       
 14  NI        6374 non-null   float64       
 15  RECT      6340 non-null   float64       
 16  REVT      6374 non-null   float64       
 17  SALE      637

#### Problem 2.7 - Excluding some industries (7 points)

When performing data analyses such as ours, many researchers and analysts exclude data for companies in the financial services, insurance, and real estate industries. Companies in these industries have very different accounting practices than other companies, and hence their accounting numbers are not comparable.

A common way to identify financial, insurance, and real estate companies is by SIC codes. SIC stands for "standard industrial classification" and is a taxonomy developed by various governments to classify companies. Go to [this website](https://siccode.com/) and determine the *range* of SIC codes that you want to exclude from our dataset. Be careful when reading this website! Ignore anything about NAICS (an alternative to SIC). Also, SIC codes are 4-digit codes. Keep that in mind as you interpret the information at the SIC website.

---

Write code to exclude financial and insurance companies from the Compustat data frame:

In [176]:
# Enter your code in this cell
compustat.drop(compustat[(compustat["SIC"]>=6000) & (compustat["SIC"]<6500)].index, inplace=True)
compustat.drop(compustat[(compustat["SIC"]>=6700) & (compustat["SIC"]<6800)].index, inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750,5080
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668,7380
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201,4512
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197,3564
17,1072,2019-03-31,2018,AVX,AVX CORP,3,1783.727,2813.278,2384.180,271.813,631.688,142.475,313.185,429.098,271.813,262.279,1791.790,1791.790,859163,3670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28023,332115,2018-12-31,2018,ARMP,ARMATA PHARMACEUTICALS INC,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,921114,2836
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147,7370
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295,3524
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250,5961


We lost many rows! Looks like there were many financial companies in our dataset.

#### Problem 2.8 - Dealing with missing values, part 2 (4 points)

Let's clean up our data some more. If a company does not report values in some critical columns (like total assets) then we should assume the data is suspect and drop that row.

Remove rows with missing values in any of these columns:
* Total assets
* Common equity
* Income before extraordinary items
* Total liabilities
* Net income
* Revenues
* Sales

In [177]:
# Enter your code in this cell
compustat.dropna(subset=["AT", "CEQ", "IB", "LT", "NI", "REVT", "SALE"], inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750,5080
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668,7380
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201,4512
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197,3564
17,1072,2019-03-31,2018,AVX,AVX CORP,3,1783.727,2813.278,2384.180,271.813,631.688,142.475,313.185,429.098,271.813,262.279,1791.790,1791.790,859163,3670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28018,330227,2018-09-30,2018,CTRM,CASTOR MARITIME INC,9,2.110,9.624,9.475,0.981,0.061,0.000,0.149,0.149,0.981,0.265,3.961,3.961,1720161,4412
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147,7370
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295,3524
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250,5961


#### Problem 2.9 - Dealing with zero and negative values (3 points)

We just removed rows with missing values in critical columns. Now let's remove rows with zero or negative values in certain columns. Zero and negative values might be legitimate, but companies with negative values in these columns are likely very different than other companies and might skew our analysis.

Drop any rows with zero or negative numbers in any of these columns:
* Total assets
* Total liabilities
* Sales

In [178]:
# Enter your code in this cell
compustat.drop(compustat[(compustat["AT"]<=0) | (compustat["LT"]<=0) | (compustat["SALE"]<=0)].index, inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750,5080
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,NaN,19.536,24.826,1.101,16.316,77.742,77.742,2668,7380
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,NaN,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201,4512
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197,3564
17,1072,2019-03-31,2018,AVX,AVX CORP,3,1783.727,2813.278,2384.180,271.813,631.688,142.475,313.185,429.098,271.813,262.279,1791.790,1791.790,859163,3670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28018,330227,2018-09-30,2018,CTRM,CASTOR MARITIME INC,9,2.110,9.624,9.475,0.981,0.061,0.000,0.149,0.149,0.981,0.265,3.961,3.961,1720161,4412
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147,7370
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295,3524
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250,5961


#### Problem 2.10 - Dealing with missing values, part 3 (3 points)

Many companies do not report certain line items. For example, retailers do not have work-in-process inventories so they do not report this. Thus, some companies will have missing values in certain columns. Let's fill in those missing values with zeroes.

In the following columns, replace missing values with zeros:
* Current assets
* Inventories
* Work-in-process inventories
* Current liabilities
* Receivables

In [179]:
# Enter your code in this cell
compustat.ACT.fillna(0, inplace=True)
compustat.INVT.fillna(0, inplace=True)
compustat.INVWIP.fillna(0, inplace=True)
compustat.LCT.fillna(0, inplace=True)
compustat.RECT.fillna(0, inplace=True)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,952.500,1517.200,905.900,84.100,589.000,14.000,357.500,611.300,7.500,258.100,2051.800,2051.800,1750,5080
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,23.793,34.031,9.205,1.101,5.026,0.000,19.536,24.826,1.101,16.316,77.742,77.742,2668,7380
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8637.000,60580.000,-169.000,1412.000,1522.000,0.000,18096.000,60749.000,1412.000,1706.000,44541.000,44541.000,6201,4512
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,160.865,392.582,178.560,-7.121,50.511,35.792,103.886,214.022,-7.121,53.225,337.339,337.339,3197,3564
17,1072,2019-03-31,2018,AVX,AVX CORP,3,1783.727,2813.278,2384.180,271.813,631.688,142.475,313.185,429.098,271.813,262.279,1791.790,1791.790,859163,3670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28018,330227,2018-09-30,2018,CTRM,CASTOR MARITIME INC,9,2.110,9.624,9.475,0.981,0.061,0.000,0.149,0.149,0.981,0.265,3.961,3.961,1720161,4412
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,698.952,764.288,-312.467,-178.028,0.000,0.000,144.905,165.902,-178.028,63.359,96.666,96.666,1640147,7370
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,100.592,174.411,47.544,-32.892,53.200,0.000,43.864,126.867,-32.892,17.566,211.813,211.813,1695295,3524
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1104.000,1193.000,-1287.000,-208.000,0.000,0.000,970.000,1104.000,-208.000,93.000,1728.000,1728.000,1822250,5961


#### Problem 2.11 - Dealing with units (3 points)

Compustat data is in millions but Audit Analytics data reports actual amounts. Since we will soon merge this data, let's put all numbers on the same scale.

Convert all Compustat columns that are in millions to actual amounts.

In [180]:
# Enter your code in this cell
compustat.ACT = compustat.ACT.apply(lambda x: x*1000000)
compustat.AT = compustat.AT.apply(lambda x: x*1000000)
compustat.CEQ = compustat.CEQ.apply(lambda x: x*1000000)
compustat.IB = compustat.IB.apply(lambda x: x*1000000)
compustat.INVT = compustat.INVT.apply(lambda x: x*1000000)
compustat.INVWIP = compustat.INVWIP.apply(lambda x: x*1000000)
compustat.LCT = compustat.LCT.apply(lambda x: x*1000000)
compustat.LT = compustat.LT.apply(lambda x: x*1000000)
compustat.NI = compustat.NI.apply(lambda x: x*1000000)
compustat.RECT = compustat.RECT.apply(lambda x: x*1000000)
compustat.REVT = compustat.REVT.apply(lambda x: x*1000000)
compustat.SALE = compustat.SALE.apply(lambda x: x*1000000)
compustat

,GVKEY,DATADATE,FYEAR,TIC,CONM,FYR,ACT,AT,CEQ,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,CIK,SIC
2,1004,2019-05-31,2018,AIR,AAR CORP,5,9.525000e+08,1.517200e+09,9.059000e+08,8.410000e+07,5.890000e+08,14000000.0,3.575000e+08,6.113000e+08,7.500000e+06,2.581000e+08,2.051800e+09,2.051800e+09,1750,5080
4,1019,2018-12-31,2018,AFAP,AFA PROTECTIVE SYSTEMS INC,12,2.379300e+07,3.403100e+07,9.205000e+06,1.101000e+06,5.026000e+06,0.0,1.953600e+07,2.482600e+07,1.101000e+06,1.631600e+07,7.774200e+07,7.774200e+07,2668,7380
7,1045,2018-12-31,2018,AAL,AMERICAN AIRLINES GROUP INC,12,8.637000e+09,6.058000e+10,-1.690000e+08,1.412000e+09,1.522000e+09,0.0,1.809600e+10,6.074900e+10,1.412000e+09,1.706000e+09,4.454100e+10,4.454100e+10,6201,4512
10,1050,2018-12-31,2018,CECO,CECO ENVIRONMENTAL CORP,12,1.608650e+08,3.925820e+08,1.785600e+08,-7.121000e+06,5.051100e+07,35792000.0,1.038860e+08,2.140220e+08,-7.121000e+06,5.322500e+07,3.373390e+08,3.373390e+08,3197,3564
17,1072,2019-03-31,2018,AVX,AVX CORP,3,1.783727e+09,2.813278e+09,2.384180e+09,2.718130e+08,6.316880e+08,142475000.0,3.131850e+08,4.290980e+08,2.718130e+08,2.622790e+08,1.791790e+09,1.791790e+09,859163,3670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28018,330227,2018-09-30,2018,CTRM,CASTOR MARITIME INC,9,2.110000e+06,9.624000e+06,9.475000e+06,9.810000e+05,6.100000e+04,0.0,1.490000e+05,1.490000e+05,9.810000e+05,2.650000e+05,3.961000e+06,3.961000e+06,1720161,4412
28028,339965,2019-01-31,2018,SNOW,SNOWFLAKE INC,1,6.989520e+08,7.642880e+08,-3.124670e+08,-1.780280e+08,0.000000e+00,0.0,1.449050e+08,1.659020e+08,-1.780280e+08,6.335900e+07,9.666600e+07,9.666600e+07,1640147,7370
28030,345920,2018-12-31,2018,HYFM,HYDROFARM HLDNG GP INC,12,1.005920e+08,1.744110e+08,4.754400e+07,-3.289200e+07,5.320000e+07,0.0,4.386400e+07,1.268670e+08,-3.289200e+07,1.756600e+07,2.118130e+08,2.118130e+08,1695295,3524
28033,345980,2018-12-31,2018,WISH,CONTEXTLOGIC INC,12,1.104000e+09,1.193000e+09,-1.287000e+09,-2.080000e+08,0.000000e+00,0.0,9.700000e+08,1.104000e+09,-2.080000e+08,9.300000e+07,1.728000e+09,1.728000e+09,1822250,5961


### Problem 3 - Merging the Audit Analytics and Compustat Datasets (15 points)

At this point, you should have two data frames, one containing cleaned Audit Analytics data, and the other containing cleaned Compustat data. See problem 1.10 for some guidance about merging.

#### Problem 3.1 - Setting up the merge (3 points)

In your Audit Analytics data frame, there is a column named `COMPANY_FKEY`. Rename it to `CIK`.

In [181]:
# Enter your code in this cell
audit_analytics.rename(columns={"COMPANY_FKEY": "CIK"}, inplace=True)
audit_analytics

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,CIK,NAME,Max_Audit_Fees
0,4,227633,2018,2018-05-31,1988901,317296,2306197,174659,0,KPMG LLP,1750,AAR CORP,1988901
1,2,230983,2018,2018-12-31,25381000,9683000,35064000,770000,8000,Ernst & Young LLP,1800,ABBOTT LABORATORIES,25381000
2,11761,228249,2018,2018-06-30,2028000,229000,2257000,0,0,BDO USA LLP,2034,ACETO CORP,2028000
3,8256,231256,2018,2018-12-31,328000,74000,402000,22000,0,Marcum LLP,2098,ACME UNITED CORP,328000
4,1,230432,2018,2018-12-31,272000,101900,373900,26500,0,PricewaterhouseCoopers LLP,2110,COLUMBIA ACORN TRUST,272000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,1,240882,2018,2018-12-31,2155000,527000,2682000,302000,5000,PricewaterhouseCoopers LLP,1792580,Ovintiv Inc.,2155000
4896,3,244483,2018,2018-12-31,2221000,206000,2427000,0,5000,Deloitte & Touche LLP,1793294,"PPD, Inc.",2221000
4897,2,244138,2018,2018-12-31,3603000,47000,3650000,0,41000,Ernst & Young LLP,1794338,International General Insurance Holdings Ltd.,3603000
4898,4,242381,2018,2018-12-31,1100000,400000,1500000,0,0,KPMG LLP,1794846,Atlas Corp.,1100000


#### Problem 3.2 - Merge the data frames (8 points)

Merge the Audit Analytics and Compustat datasets using the CIK column.

Hints: 
* We strongly recommend that you use the Pandas merge function [pd.merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html).
* Use an inner join.
* Merge on the CIK column.
* Use the Audit Analytics data frame as the "left" and Compustat as the "right".

In [182]:
# Enter your code in this cell
res: pd.DataFrame = pd.merge(left=audit_analytics, right=compustat, how="inner", on="CIK")
res

,AUDITOR_FKEY,AUDIT_GIG_KEY,FISCAL_YEAR,FISCAL_YEAR_ENDED,AUDIT_FEES,NON_AUDIT_FEES,TOTAL_FEES,AUDIT_RELATED_FEES,OTHER_FEES,AUDITOR_NAME,...,IB,INVT,INVWIP,LCT,LT,NI,RECT,REVT,SALE,SIC
0,4,227633,2018,2018-05-31,1988901,317296,2306197,174659,0,KPMG LLP,...,8.410000e+07,5.890000e+08,14000000.0,3.575000e+08,6.113000e+08,7.500000e+06,2.581000e+08,2.051800e+09,2.051800e+09,5080
1,2,230983,2018,2018-12-31,25381000,9683000,35064000,770000,8000,Ernst & Young LLP,...,2.334000e+09,3.796000e+09,499000000.0,9.012000e+09,3.645100e+10,2.368000e+09,5.182000e+09,3.057800e+10,3.057800e+10,3845
2,11761,228249,2018,2018-06-30,2028000,229000,2257000,0,0,BDO USA LLP,...,-3.161210e+08,1.370760e+08,0.0,3.025180e+08,6.717390e+08,-3.161210e+08,2.569100e+08,7.113590e+08,7.113590e+08,5160
3,8256,231256,2018,2018-12-31,328000,74000,402000,22000,0,Marcum LLP,...,4.598000e+06,4.133200e+07,142000.0,1.336500e+07,5.714500e+07,4.598000e+06,2.510200e+07,1.373210e+08,1.373210e+08,3420
4,4,232503,2018,2018-12-31,864000,60000,924000,10000,0,KPMG LLP,...,2.945000e+06,2.277900e+07,0.0,1.232670e+08,1.322720e+08,2.945000e+06,8.802600e+07,1.750213e+09,1.750213e+09,5172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,4,244043,2018,2018-12-31,3008747,492616,3501363,394093,22736,KPMG LLP,...,-1.737380e+08,3.680210e+08,0.0,1.017056e+09,4.184174e+09,-1.737380e+08,4.903320e+08,3.520933e+09,3.520933e+09,1090
3196,2,239755,2018,2018-12-31,1285000,0,1285000,0,0,Ernst & Young LLP,...,7.080000e+07,0.000000e+00,0.0,1.024000e+08,1.088000e+08,7.080000e+07,6.300000e+07,6.624000e+08,6.624000e+08,1311
3197,1,240882,2018,2018-12-31,2155000,527000,2682000,302000,5000,PricewaterhouseCoopers LLP,...,1.069000e+09,0.000000e+00,0.0,2.016000e+09,7.897000e+09,1.069000e+09,1.005000e+09,5.939000e+09,5.939000e+09,1311
3198,3,244483,2018,2018-12-31,2221000,206000,2427000,0,5000,Deloitte & Touche LLP,...,1.041860e+08,0.000000e+00,0.0,1.794673e+09,6.986890e+09,1.041860e+08,1.276789e+09,3.748971e+09,3.748971e+09,8731


#### Problem 3.3 - Save to Excel (4 points)

In the next step, you will need to load your data into Tableau for analysis. Therefore, save the merged data frame to an Excel file. Use the keyword argument `index=False` when you save to Excel. That will tell Pandas not to write the index to the Excel file and will make it easier to work with the data in Tableau.

In [183]:
# Enter your code in this cell
res.to_excel("data/res.xlsx", index=False)

### Problem 4 Analyzing the Data (45 points)

We will use Tableau for our analysis. Load the Excel file that you just saved into Tableau.

#### Problem 4.1 - How many clients audited by each firm? (5 points)

Make a bar chart showing the number of clients audited by each auditing firm. 
* The bars should be horizontal.
* Sort the bars in descending order.
* Label each bar with the number of clients audited by that firm.
* Name the worksheet "Problem 4.1"

---

In the caption for this worksheet, answer these questions:
1. Are the "Big 4" actually the "Big 4"? Do these 4 firms have the most clients?
2. Who are the fifth and sixth largest firms, by number of clients?
3. What is the market share of the Big 4 (in total), as measured by number of clients? In other words, what percentage of companies in the dataset are audited by the Big 4? You might need to do this with a calculator, or in Python.

#### Problem 4.2 - Total audit fees collected by each firm (5 points)

Make a bar chart showing the total audit fees collected by each auditing firm. 
* The bars should be horizontal.
* Sort the bars in descending order.
* Label each bar with the total fees collected by that firm.
* Format the labels in currency format, with zero decimal places.
* Name the worksheet "Problem 4.2"

---

In the caption for this worksheet, answer these questions:
1. Does the firm with the most clients collect the most fees?
2. What is the market share of the Big 4, as measured by revenues? In other words, what percentage of total audit revenue in the dataset is collected by the Big 4? You might need to do this with a calculator, or in Python.

#### Problem 4.3 - Average audit fees collected by each firm (5 points)

Make a bar chart showing the *average* audit fees per client collected by each auditing firm. 
* The bars should be horizontal.
* Sort the bars in descending order.
* Label each bar with the average fees collected by that firm.
* Format the labels in currency format, with zero decimal places.
* Name the worksheet "Problem 4.3"

---

In the caption for this worksheet, answer these questions:
1. Are Grant Thornton and BDO still in 5th and 6th position? 
2. Speculate on how an auditing firm like *Macias, Gini, & O'Connell*, with relatively few clients, might collect such high revenues per client.

#### Problem 4.4 - Company size as a driver of audit fees, part 1

Let's begin to examine the drivers of audit fees. A common finding in accounting research is that audit fees are higher for larger companies. This makes sense. Amazon should pay more to their auditor than a small company. If we do not find that company size is positively correlated with audit fees, we would be surprised and would question the integrity of our data. In this project, we will measure a company's size by its total assets. This is a common measure in research; another common measure is revenues.

Even though this analysis may sound straightforward, you are about to see that there are some challenges when working with this data. In this problem, we will examine audit fees and company size (as measured by total assets). In the next problem, we will check whether they are related.

##### Problem 4.4 (a) -- (5 points)
Make a histogram of audit fees in Tableau. You may use the default bin size. Name the worksheet "Problem 4.4 (a)".

---

Answer these questions in the caption:
1. You have seen histograms like this before. How do you interpret this histogram? What does it tell you about your audit fee data?
2. What is the range of fees in the tallest bin?
3. Is this histogram what you expect? In other words, does the shape of this histogram surprise you? Why or why not?

##### Problem 4.4 (b) -- (3 points)
A common measure of company size is total assets. Make a histogram of total assets in Tableau. You may use the default bin size. Name the worksheet "Problem 4.4 (b)".

---

Answer this question in the caption: is this histogram consistent with the previous one?

##### Problem 4.4 (c) -- (4 points)
Filter the histogram from 4.4(b) so that it only includes companies with total assets of less than \\$10 billion. Set the bin size to \\$100 million. Name the worksheet "Problem 4.4 (c)".

Be careful! Did your histogram for 4.4(b) change? If so, fix it so the histograms for parts b and c are independent.

---

Answer these questions in the caption: 
1. Does the pattern from 4.4(b) persist?
2. How many companies are in the first bin?
3. What is the range of the first bin?

#### Problem 4.5 - Company size as a driver of audit fees, part 2

In this problem, we will investigate the correlation between audit fees and company size (as measured by total assets).

##### Problem 4.5 (a) -- (3 points)
* Make a scatter plot of audit fees (y-axis) versus total assets (x-axis) in Tableau. 
* Add a trend line.
* Make the markers as small as possible.
* Name the worksheet "Problem 4.5 (a)".

##### Problem 4.5 (b) -- (8 points)
The scatter plot in 4.5(a) reveals an upward trend, but the graph is messy. The trend line has a good fit (R-squared is above 0.5), but the large range of the data makes it difficult to see a pattern. Also, there is a significant overplotting problem.

Throughout the semester, when we encountered data with large ranges like this, we filtered it to a small range. However, doing so here would be problematic since such filtering would remove many "blue chip" clients and disproportionately impact our inferences about the Big 4, who tend to audit large companies.

A common way of dealing with data with large ranges is to change the scale of the axes. Let's do that and then understand why it is helpful.

---

In Tableau:
* Duplicate the worksheet titled "Problem 4.5 (a)". When you do, you will see a new worksheet titled "Problem 4.5 (a) (2)". Rename that to "Problem 4.5 (b)".
* Remove the trend line.
* Right-click on the x-axis and click **Edit Axis...** Check the box next to the word *Logarithmic*.
* Repeat the previous step for the y-axis (in other words, make the y-axis logarithmic).

---

Look at that! The data looks very different! It is the exact same data. We have not changed the data in any way. We are just displaying it differently. Look closely at the tick marks for the x-axis. They should be powers of 10 (e.g., 1, 100, 1000). On a logarithmic scale, every tick mark on an axis is 10 times larger than the last tick mark! Logarithmic scales are often used when working with data that has a wide range as it allows us to easily visualize such data. We have solved the clustering and overplotting problems in the previous problem. 

In the caption, answer these questions:
1. Does the graph show a relationship between fees and assets?
2. Does the relationship appear "stronger" than that in problem 4.5(a)?

---

Now do the following:
* Add a trend line to this graph. When you do, the trend line might look weird, like a hockey stick. That's because a line, when plotted on a logarithmic scale, no longer appears to be a line.
* Click on the trend line and click Edit.
* You will see a list of options: linear, logarithmic, exponential, power, polynomial. Try each one. Choose the one that looks the best to you.

---

If you chose the Power option, your trend line will look like a line. If you hover over the trend line, you will see its equation. The equation will look like $Audit Fees = a \cdot {AT}^b$, where $a$ and $b$ are numbers. 

What this tells us is that fees equal a constant times assets raised to a power. The exponent should be close to 0.5, so basically we are taking the square root of assets. In other words, as assets go up, fees increase by the square root of assets. Note that our trend line equation is <i><u>not</u></i> a linear equation. Instead, it shows that as company size increases, the rate at which fees increase diminishes and levels off. This makes economic sense. As a company grows from small to mid-size, the complexity of the audit will increase dramatically. As a large company (like Apple) grows, the extra work done by the auditor will grow, but not as much as it would from a small to a medium company. The auditor will benefit from an economy of scale.

---

In the caption, answer this question:  
3. Use the trend line equation to predict the audit fees for a company with \\$1 billion in total assets. You may need to use a calculator for this.

#### Problem 4.6 - Risk as a driver of audit fees (7 points)

In this problem, we will investigate the relationship between audit fees and the risk of an audit engagement. Whenever an auditor agrees to audit a client, they assume some risk. If the auditor does not catch mistakes or fraud, investors can sue the auditor. Accounting research has found that inventory is a measure of risk. The reason is that it takes effort to audit inventories; often, physical counts must be taken in multiple locations and then reconciled with the client's accounting records. This process dramatically increases the auditor's workload and hence audit fees. Let's confirm this hypothesis in our data.

Many companies, such as those that provide services, do not have inventory. Or if they do, inventory is immaterial and reported as 0. Therefore, let's focus on two industries that have significant inventory: retail and manufacturing. 

---

In Tableau:
* Create a "calculated field" called "Industry". 
    - The value of this field should be "Retail" if the SIC code for a row matches:
        * "Building Materials, Hardware, Garden Supplies, and Mobile Home Dealers"
        * "General Merchandise Stores"
        * "Food Stores"
        * "Automotive Dealers and Gasoline Service Stations"
        * "Apparel and Accessory Stores"
        * "Home Furniture, Furnishings, and Equipment Stores"
    - The value of this field should be "Manufacturing" if the SIC code for a row is in the range of "Manufacturing". 
    - The value of this field should be "Other" for all other SIC codes. 
    - In other words, the "Industry" field can only have 3 values, "Retail", "Manufacturing", or "Other".
    - Look [here](https://siccode.com/) for SIC codes.
* Now create two scatter plots, on the same worksheet, of Audit Fees (y-axis) versus Inventory (x-axis).
    - There should be one graph for Manufacturing and one for Retail. They should be stacked (Manufacturing on top of Retail).
    - Do NOT show a graph for "Other" industries.
    - Make the markers as small as possible.
    - Change the scale for both axes to logarithmic
    - Add trend lines to both graphs; use the Power option.
    - Name this worksheet "Problem 4.6"
    
---

You have two trend lines, one for manufacturing and one for retail. Use the equations for the two trend lines to answer these questions in the worksheet caption. Remember that you can see the equations for the trend linse by hovering over them with your mouse.

1. Use the trend line from the manufacturing graph to calculate the predicted audit fees for a manufacturer with inventories of \\$1 billion. You may need to use a calculator.
2. Use the trend line from the retail graph to calculate the predicted audit fees for a retailer with inventories of \\$1 billion. You may need to use a calculator.
3. Compare the two numbers and speculate on why one is higher than the other.

## Part 2 - Auditor Tenure (35 points)

Auditor tenure is the length of time for which an auditor has audited a client company. Some people argue that longer auditor tenure can lead to a less independent auditor because the auditor has developed a relationship with their client over time. Indeed, audit partners must rotate off of their public audit clients in order to avoid any independence concerns. Others have investigated this possibility and found that auditor tenure is not associated with lower quality audits. The debate about auditor tenure continues. 

In this problem you will use annual reports to determine the auditor and auditor tenure for each company. The Excel file *GatherAuditorData.xslx* contains a list of the top 20 companies in the Fortune 500. Open that file in Excel now and scan it. You will see 6 columns:

* Rank
* Company
* Ticker
* AuditorName
* AuditorSince
* NumYearsAsAuditor

The columns `AuditorName`, `AuditorSince`, and `NumYearsAsAuditor` are empty and you will populate them. To do this, you will write code that opens each annual report (PDFs), extracts the text, searches the text for relevant information, and adds this information to a dataframe.

### Problem 5.1 - Read the Excel file (3 points)

Read the Excel file *GatherAuditorData.xlsx* into a data frame. Name the data frame `df`.

In [184]:
# Enter your code in this cell
df: pd.DataFrame = pd.read_excel("data/GatherAuditorData.xlsx")
df

,Rank,Company,Ticker,AuditorName,AuditorSince,NumYearsAsAuditor
0,1,Walmart,WMT,NaN,NaN,NaN
1,2,Amazon,AMZN,NaN,NaN,NaN
2,3,Apple,AAPL,NaN,NaN,NaN
3,4,CVS Health,CVS,NaN,NaN,NaN
4,5,UnitedHealth Group,UNH,NaN,NaN,NaN
5,6,Exxon Mobil,XOM,NaN,NaN,NaN
6,7,Berkshire Hathaway,BRK,NaN,NaN,NaN
7,8,Alphabet,GOOGL,NaN,NaN,NaN
8,9,McKesson,MCK,NaN,NaN,NaN
9,10,AmerisourceBergen,ABC,NaN,NaN,NaN


Now run the following cell to populate the empty `AuditorName` column with empty strings. This will prevent errors later in this problem.

If your data frame has a different name than df, please modify the cell below before running it.

In [185]:
df['AuditorName'] = ''
df

,Rank,Company,Ticker,AuditorName,AuditorSince,NumYearsAsAuditor
0,1,Walmart,WMT,,NaN,NaN
1,2,Amazon,AMZN,,NaN,NaN
2,3,Apple,AAPL,,NaN,NaN
3,4,CVS Health,CVS,,NaN,NaN
4,5,UnitedHealth Group,UNH,,NaN,NaN
5,6,Exxon Mobil,XOM,,NaN,NaN
6,7,Berkshire Hathaway,BRK,,NaN,NaN
7,8,Alphabet,GOOGL,,NaN,NaN
8,9,McKesson,MCK,,NaN,NaN
9,10,AmerisourceBergen,ABC,,NaN,NaN


### Problem 5.2 - Extract Auditor info, populate data frame (32 points)

There are 20 PDFs that accompany this project, one for each company listed in the Excel file. The names of the PDFs correspond to the ticker symbols of the companies in the Excel file. Write a loop that iterates over these PDFs and does the following:

For each PDF:
1. Open the PDF file
2. Extract the text for all pages
3. Within the text of each PDF, there is a sentence in the auditor's letter that states the year in which the auditor began auditing that company. The sentence is structured differently in every file, but it always ends with "*auditor since XXXX*", where *XXXX* is a 4-digit year. Use this knowledge to search the PDF for this piece of text. Extract the year, and use it to complete the columns `AuditorSince` and `NumYearsAsAuditor`.
4. Search each annual report for the names of the Big 4 auditors and add the auditor name to the `AuditorName` column.
5. Convert the `AuditorSince` and `NumYearsAsAuditor` columns of the data frame to integer.
6. Ensure the resulting dataframe is sorted by rank and display/print the data frame so we can see the result.

Hints:
* Use the `PyPDF2` library.
* Use the Python regular expressions library, `re`.
* The Python regular expressions library has a method `re.findall` that returns a list of all matches of a regular expression within a string.
* To update data in a single cell in your dataframe, use the `.at` method.
* Consider setting the dataframe index to the Ticker column. Note that this is not mandatory. It is possible to do this problem without doing this.

Finally, note that this task might take some time to run on your computer. Don't be surprised if it takes 10 minutes to go through all 20 PDF files.

In [186]:
# Enter your code in this cell
import PyPDF2
import os
import re

data_dir: str = "data"
paths: list[str] = []
for relpath, dirs, files in os.walk(data_dir):
    for file in files:
        if "pdf" in file:
            paths.append(os.path.join(relpath, file))

for path in paths:
    pdf_file = PyPDF2.PdfFileReader(open(path, "rb"))
    text: str = ""
    for p in range(pdf_file.getNumPages()):
        text += pdf_file.getPage(p).extractText()
    ticker: str = re.match(r"data/(.+)\.pdf", path).group(1)
    year: str = re.search(r"auditor since ([0-9]{4})", text).group(1)
    df.loc[df.Ticker==ticker, "AuditorSince"] = year
    df.loc[df.Ticker==ticker, "NumYearsAsAuditor"] = year
    auditor: str = ""
    try:
        auditor = re.search(r"/[Ss]/\s+(.+)\s+LLP", text).group(1)
    except AttributeError:
        if text.find("PricewaterhouseCoopers") != -1:
            auditor = "PricewaterhouseCoopers"
        elif text.find("Ernst & Young") != -1:
            auditor = "Ernst & Young"
    df.loc[df.Ticker==ticker, "AuditorName"] = auditor
df[["AuditorSince"]] = df[["AuditorSince"]].astype(np.int64)
df[["NumYearsAsAuditor"]] = df[["NumYearsAsAuditor"]].astype(np.int64)
df.sort_values(by="Rank", inplace=True)
df = df.set_index(["Ticker"])
df

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


,Rank,Company,AuditorName,AuditorSince,NumYearsAsAuditor
Ticker,,,,,
WMT,1,Walmart,Ernst & Young,1969,1969
AMZN,2,Amazon,Ernst & Young,1996,1996
AAPL,3,Apple,Ernst & Young,2009,2009
CVS,4,CVS Health,Ernst & Young,2007,2007
UNH,5,UnitedHealth Group,DELOITTE & TOUCHE,2002,2002
XOM,6,Exxon Mobil,PricewaterhouseCoopers,1934,1934
BRK,7,Berkshire Hathaway,Deloitte & Touche,1985,1985
GOOGL,8,Alphabet,Ernst & Young,1999,1999
MCK,9,McKesson,Deloitte & Touche,1968,1968
